In [1]:
import numpy as np

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit.quantum_info.operators import Operator

In [2]:
kappa = 0.5
delta_t = 1

# DnQm
# n, m = 3, 4
n, m = 3, 18

# NX x NY x NZ
NX = 100
NY = 100
NZ = 100

# Identity matrices for reuse
I_X = np.eye((NX))
I_Y = np.eye((NY))
I_Z = np.eye((NZ))
I = np.kron(I_Z, np.kron(I_Y, I_X))

print(I)

MemoryError: Unable to allocate 7.28 TiB for an array with shape (100, 10000, 100, 10000) and data type float64

In [3]:
n_qubits_switch = 2
n_qubits_direction = int(np.ceil(np.log2(m)))
n_qubits_grid = int(np.ceil(np.log2(NX * NY * NZ)))
n_qubits_ancilla = 5
n_qubits = n_qubits_switch + n_qubits_direction + n_qubits_grid + n_qubits_ancilla

print(n_qubits)

32


In [4]:
# initial statevector preparation

# initial intensities (not including sources)
I_i = np.zeros(shape=(NX*NY*NZ, m))

# initial sources
S_i = np.zeros(shape=(NX*NY*NZ, m))
S_i[0, :] = 0.1 # @TODO - also test with all zeros

# initial grid state preparation
# the initial state has the following classical form:
#     (I_0(0,0,0), ..., I_m(NX-1, NY-1, NZ-1))
#   + delta_t/2 (S_0(0, 0, 0), ..., S_m(NX-1, NY-1, NZ-1))
def state_preparation_3d(qc):
    initial_statevector = np.zeros((2**n_qubits))

    coordinate_max = NX * NY * NZ
    coordinate_max_bin = bin(coordinate_max)[2:]
    for coordinate_i in range(coordinate_max):
        coordinate_bin = bin(coordinate_i)[2:].zfill(len(coordinate_max_bin))

        m_max_bin = bin(m)[2:]
        for m_i in range(m):
            m_bin = bin(m_i)[2:].zfill(len(m_max_bin))
        
            for s in range(2):
                c = 0
                
                if s == 0:
                    c = I_i[coordinate_i, m_i]
                else:
                    c = 0.5 * delta_t * S_i[coordinate_i, m_i]
                
                idx_bin = f"0b{s}{m_bin}{coordinate_bin}"
                idx_dec = int(idx_bin, 2)
                initial_statevector[idx_dec] = c

                # if c > 0:
                #     print(idx_bin, idx_dec, c)

                # if idx_dec == 4:
                #     print(s, m_bin, coordinate_bin, idx_bin, idx_dec, c)

    # print(initial_statevector)
    # print(np.sum(initial_statevector), np.linalg.norm(initial_statevector))
    initial_statevector /= np.linalg.norm(initial_statevector)
    
    qc.initialize(initial_statevector)

In [5]:
# absorption and scattering operation
def absorption_scattering_3d(qc):
    D = np.eye((m))

    A = np.kron(I, D)

    print("D:", D, np.shape(D))
    print("A:", A, np.shape(A))

    C_1 = A + 0.5j * np.sqrt(1 - A**2)
    C_2 = A - 0.5j * np.sqrt(1 - A**2)

    U_C1 = Operator(C_1)
    U_C2 = Operator(C_2)

    print(U_C1.to_matrix())
    print(U_C2.to_matrix())
    
    # qc.unitary(U_C1, list(range(n_qubits)), label='U_C1')
    # qc.unitary(U_C2, list(range(n_qubits)), label='U_C2')

In [6]:
qc = QuantumCircuit(n_qubits)

state_preparation_3d(qc)
absorption_scattering_3d(qc)

qc.draw(output="mpl")

MemoryError: Unable to allocate 32.0 GiB for an array with shape (4294967296,) and data type float64

In [7]:
import numpy as np
import pennylane as qml

a = 0.25

# matrix to be decomposed
A = a * np.array(
    [[1, 0, 0, 0],
     [0, 1, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 0, 1]]
)

LCU = qml.pauli_decompose(A)
LCU_coeffs, LCU_ops = LCU.terms()

print(f"LCU decomposition:\n {LCU}")
print(f"Coefficients:\n {LCU_coeffs}")
print(f"Unitaries:\n {LCU_ops}")

ModuleNotFoundError: No module named 'pennylane'